## Deep Convolutional MNIST
#### 이 문서는 [tensorflow tutorial](http://www.tensorflow.org/tutorials/mnist/pros/index.html)을 바탕으로 작성되었습니다.
이 MNIST를 한번도 안다뤄봤거나 deep-learning이 낯설다면 [MNIST Basic](TensorFlow-MNIST.ipynb)을 먼저 보시는 것을 추천합니다.

In [1]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


일반적으로 tensorflow는 graph를 만든 후에 session을 동작시키는 방식으로 동작한다. 하지만 여기선 학습을 위해 InteractiveSession을 사용하도록 하자. InteractiveSession은 iPython과 같이 동작한다.

In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

### Placeholders
placeholder는 실제 데이터가 아닌 데이터가 들어갈 공간을 미리 정의해놓은 곳이라고 보면 되겠다. 여기서는 784길이의 입력벡터, 10길이의 출력 벡터를 정의한다.

In [8]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

### Variables
업데이트할 파라메터들을 variable로 정의하면 된다. Variables는 session에서 사용되기전에 반드시 초기화되어야 한다. 실제로 값을 집어 넣는 명령어라고 볼 수 있겠다.

In [11]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.initialize_all_variables())

### Implement model and Cost Function
모델과 cost function을 정의한다. 여기선 softmax와 cross entropy를 사용했다.

In [13]:
y = tf.nn.softmax(tf.matmul(x, W) + b)
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

### Train the Model
어떤 방법으로 훈련할 것인지를 정의해야 한다. 여기선 Gradient Descent방법을 사용한다. 훈련 방법을 적을 땐 learning rate와 cost function을 인자로 전달한다. 여기선 훈련 방법을 정의할 뿐 실제로 훈련을 수행하진 않는다. 실제로 파라메터를 업데이트하기 위해서 run()을 수행한다. feed_dict은 placeholder tensor에 실제 값을 입력한다.

In [15]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

### Evaluate the Model
검증할 때에도 검증할 함수를 미리 정의해놓고 session을 돌리는 방식으로 하면 된다.

In [16]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})

0.9092


*여기까진 이전 튜토리얼과 똑같다.*

## Build a Multilayer Convolutional Network
99%를 위해 달려보자

### Weight Initialization
initialize는 중요하다. 매우.. 앞으로 빈번하게 사용될 함수니 미리 정의해놓자.

In [18]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Convolution and Pooling
아쉽지만 이전 튜토리얼과 같이 기술적인 내용에 대한 보충설명은 없다. Convoluation과 Pooling이 뭔지 궁금한 분은 [구글링](https://www.google.co.kr/search?client=ubuntu&channel=fs&q=cnn&ie=utf-8&oe=utf-8&gfe_rd=cr&ei=sqNaVv_nEOf98wfHoLiQDA#channel=fs&q=cnn+dnn+convolutional).. 가물가물한 분은 아래 그림을 참조(input image가 그림에선 32x32로 표현된 것을 빼고는 같다.)
<img src="png/cnn_architecture.jpg" width=100%>
tensorflow에선 convoluation과 pooling을 할 때 여러 옵션을 줄 수 있다. 여기선 간단히 연산 후 벡터크기가 같도록 zero padding을 하고 2x2 max pooling을 사용했다.

In [19]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

### First Convoluational Layer


In [20]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Second Convolutional Lyaer

In [21]:
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Densely Connected Layer

In [23]:
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout

In [24]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout Layer

In [25]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### Train and Evaluate the Model

In [26]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict = {
                x:batch[0], y_:batch[1], keep_prob: 1.0
            })
        print "step %d, training accuracy %g"%(i, train_accuracy)
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
print "test accuracy %g"%accuracy.eval(feed_dict={
        x:mnist.test.images, y_:mnist.test.labels, keep_prob: 1.0
    })

step 0, training accuracy 0.06
step 100, training accuracy 0.88
step 200, training accuracy 0.88
step 300, training accuracy 0.92
step 400, training accuracy 0.96
step 500, training accuracy 0.9
step 600, training accuracy 1
step 700, training accuracy 0.94
step 800, training accuracy 0.98
step 900, training accuracy 0.96
step 1000, training accuracy 0.94
step 1100, training accuracy 0.96
step 1200, training accuracy 0.94
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 1
step 1600, training accuracy 0.98
step 1700, training accuracy 0.94
step 1800, training accuracy 0.94
step 1900, training accuracy 0.98
step 2000, training accuracy 0.96
step 2100, training accuracy 1


KeyboardInterrupt: 

노트북으로는 여기까지가 한계이군..